
# Capítulo 7 — Não Supervisionado + PCA
**Objetivos:**
- `KMeans` (elbow e silhouette) e `DBSCAN` (noções)
- `PCA`: variância explicada, projeção 2D


In [ ]:

import numpy as np, matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

data = load_wine()
X = StandardScaler().fit_transform(data.data)

# Elbow para K
ks, inertias = [], []
for k in range(2, 9):
    km = KMeans(n_clusters=k, n_init=10, random_state=42).fit(X)
    ks.append(k); inertias.append(km.inertia_)
plt.figure(); plt.plot(ks, inertias, marker="o")
plt.title("Elbow (KMeans)"); plt.xlabel("k"); plt.ylabel("Inertia"); plt.show()

# Silhouette
sils = []
for k in range(2, 9):
    km = KMeans(n_clusters=k, n_init=10, random_state=42).fit(X)
    sils.append(silhouette_score(X, km.labels_))
plt.figure(); plt.plot(ks, sils, marker="s")
plt.title("Silhouette (KMeans)"); plt.xlabel("k"); plt.ylabel("Score"); plt.show()

# PCA 2D
pca = PCA(n_components=2)
Xp = pca.fit_transform(X)
km = KMeans(n_clusters=3, n_init=10, random_state=42).fit(X)
plt.figure(); plt.scatter(Xp[:,0], Xp[:,1], c=km.labels_)
plt.title(f"PCA2 + KMeans (var explicada: {pca.explained_variance_ratio_.sum():.2f})")
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.show()

# DBSCAN (demonstração)
db = DBSCAN(eps=1.8, min_samples=5).fit(Xp)
plt.figure(); plt.scatter(Xp[:,0], Xp[:,1], c=db.labels_)
plt.title("DBSCAN sobre PCA2 (labels: -1=ruído)"); plt.show()
